In [151]:
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import StratifiedKFold

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
import os

from scipy.stats import chi2_contingency
from scipy.stats import f_oneway

In [152]:
from google.colab import drive
drive.mount('/content/drive')
df_full = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Data Mining/Data/df2022_WiClass.csv')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [153]:
Filtered = ['SEXVAR', 'HHADULT', 'PHYSHLTH', 'MENTHLTH', 'POORHLTH', 'PRIMINSR', 'MEDCOST1', 'EXERANY2', 'LASTDEN4', 'RMVTETH4', 'CVDSTRK3', 'ASTHMA3', 'CHCCOPD3', 'ADDEPEV3', 'HAVARTH4', 'EDUCA', 'RENTHOM1', 'EMPLOY1', 'INCOME3', 'COVIDVA1', 'COVACGET', 'CNCRAGE', 'CSRVPAIN', 'PSATEST1', 'PSATIME1', 'PCSTALK1', 'CRGVHRS1', 'CRGVALZD', 'ACEDEPRS', 'ACEDRUGS', 'ACEHURT1', 'ACESWEAR', 'ACETOUCH', 'ACETTHEM', 'ACEHVSEX', 'ACEADSAF', 'ACEADNED', 'LSATISFY', 'EMTSUPRT', 'SDHISOLT', 'SDHEMPLY', 'FOODSTMP', 'SDHFOOD1', 'SDHBILLS', 'SDHUTILS', 'SDHUTILS', 'SDHTRNSP', 'SDHSTRE1', 'MARIJAN1', 'MENTCIGS', 'MENTECIG', 'HEATTBCO', 'ASBIBING', 'ASBIADVC', 'ASBIRDUC', 'FIREARM5', 'GUNLOAD', 'RCSXBRTH', 'CASTHNO2', 'PFPPRVN4', 'TYPCNTR9', 'BRTHCNT4', 'WHEREGET', 'NOBCUSE8', 'RRTREAT', 'RRHCARE4', 'RRPHYSM2', '_RFHLTH', '_PHYS14D', '_MENT14D', '_TOTINDA', '_EXTETH3', '_ALTETH3', '_DENVST3', '_LTASTH1', '_CASTHM1', '_ASTHMS1', '_DRDXAR2', '_RACEG22', '_AGEG5YR', '_AGE65YR', '_AGE_G', 'WTKG3', '_EDUCAG', '_INCOMG1', '_RFMAM22', '_MAM5023', '_SMOKER3', '_RFSMOK3', '_PACKDAY', 'DRNKANY6', '_DRNKWK2', '_AIDTST4', 'Class']
df1 = df_full[Filtered]
df1.head()

,SEXVAR,HHADULT,PHYSHLTH,MENTHLTH,POORHLTH,PRIMINSR,MEDCOST1,EXERANY2,LASTDEN4,RMVTETH4,...,_INCOMG1,_RFMAM22,_MAM5023,_SMOKER3,_RFSMOK3,_PACKDAY,DRNKANY6,_DRNKWK2,_AIDTST4,Class
0,1.0,NaN,88.0,88.0,NaN,3.0,2.0,1.0,1.0,1.0,...,7.0,NaN,NaN,4.0,1.0,NaN,1.0,2.300000e+01,2.0,Mild CD
1,2.0,NaN,88.0,88.0,NaN,88.0,2.0,1.0,2.0,1.0,...,5.0,2.0,2.0,1.0,2.0,0.75,2.0,5.397605e-79,2.0,No CD
2,1.0,NaN,4.0,77.0,4.0,3.0,2.0,1.0,1.0,8.0,...,6.0,NaN,NaN,4.0,1.0,NaN,1.0,2.300000e+01,2.0,Mild CD
3,1.0,NaN,88.0,88.0,NaN,3.0,2.0,1.0,1.0,1.0,...,9.0,NaN,NaN,3.0,1.0,1.00,2.0,5.397605e-79,2.0,No CD
4,1.0,NaN,1.0,3.0,88.0,2.0,2.0,1.0,1.0,8.0,...,4.0,NaN,NaN,4.0,1.0,NaN,1.0,4.700000e+01,1.0,No CD


In [154]:
# Counting missing values in each column
missing_values = df1.isnull().sum()

# Printing the top 10 columns with the highest number of missing values
top_missing_values = missing_values.nlargest(20)
print(top_missing_values)


MENTECIG    63403
NOBCUSE8    63373
RCSXBRTH    63333
CASTHNO2    63025
GUNLOAD     62966
WHEREGET    62949
BRTHCNT4    62853
TYPCNTR9    62851
PFPPRVN4    62570
MENTCIGS    62514
FIREARM5    61952
PSATIME1    61903
HEATTBCO    61841
PCSTALK1    61119
PSATEST1    60940
COVACGET    60523
CRGVALZD    59382
CRGVHRS1    58710
ASBIRDUC    56659
CSRVPAIN    55822
dtype: int64


In [155]:
filtered_lav = ['SEXVAR', 'HHADULT', 'PHYSHLTH', 'MENTHLTH', 'POORHLTH', 'PRIMINSR', 'MEDCOST1', 'EXERANY2', 'LASTDEN4', 'RMVTETH4', 'CVDSTRK3', 'ASTHMA3', 'CHCCOPD3', 'ADDEPEV3', 'HAVARTH4', 'EDUCA', 'RENTHOM1', 'EMPLOY1', 'INCOME3', 'Class']
df = df_full[filtered_lav]
df.head(20)

,SEXVAR,HHADULT,PHYSHLTH,MENTHLTH,POORHLTH,PRIMINSR,MEDCOST1,EXERANY2,LASTDEN4,RMVTETH4,CVDSTRK3,ASTHMA3,CHCCOPD3,ADDEPEV3,HAVARTH4,EDUCA,RENTHOM1,EMPLOY1,INCOME3,Class
0,1.0,NaN,88.0,88.0,NaN,3.0,2.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,6.0,1.0,7.0,11.0,Mild CD
1,2.0,NaN,88.0,88.0,NaN,88.0,2.0,1.0,2.0,1.0,2.0,2.0,2.0,2.0,1.0,6.0,1.0,2.0,7.0,No CD
2,1.0,NaN,4.0,77.0,4.0,3.0,2.0,1.0,1.0,8.0,2.0,2.0,2.0,1.0,1.0,6.0,1.0,7.0,9.0,Mild CD
3,1.0,NaN,88.0,88.0,NaN,3.0,2.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,1.0,6.0,1.0,7.0,99.0,No CD
4,1.0,NaN,1.0,3.0,88.0,2.0,2.0,1.0,1.0,8.0,2.0,2.0,2.0,2.0,2.0,6.0,1.0,7.0,6.0,No CD
5,2.0,NaN,88.0,88.0,NaN,3.0,2.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,1.0,5.0,1.0,7.0,6.0,No CD
6,2.0,NaN,3.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,4.0,1.0,2.0,7.0,No CD
7,2.0,NaN,88.0,88.0,NaN,3.0,2.0,1.0,4.0,3.0,2.0,2.0,2.0,2.0,1.0,4.0,1.0,8.0,2.0,No CD
8,2.0,NaN,1.0,88.0,88.0,3.0,2.0,1.0,1.0,8.0,2.0,2.0,2.0,2.0,1.0,6.0,1.0,7.0,6.0,No CD
9,1.0,NaN,88.0,3.0,88.0,3.0,2.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,1.0,5.0,1.0,7.0,6.0,No CD


In [156]:
# Count blank values in each column
blank_values = df.isnull().sum()

# Print the number of blank values in each column
print("Number of blank values in each column:")
print(blank_values)


Number of blank values in each column:
SEXVAR          0
HHADULT     21574
PHYSHLTH        0
MENTHLTH        0
POORHLTH    29485
PRIMINSR        0
MEDCOST1        0
EXERANY2        0
LASTDEN4        0
RMVTETH4        0
CVDSTRK3        0
ASTHMA3         0
CHCCOPD3        0
ADDEPEV3        0
HAVARTH4        0
EDUCA           0
RENTHOM1        0
EMPLOY1         0
INCOME3         1
Class           0
dtype: int64


In [157]:

# Impute missing values in POORHLTH and HHADULT columns with median values
median_poorhlth = df['POORHLTH'].median()
median_hhadult = df['HHADULT'].median()
median_income = df['INCOME3'].median()


df['POORHLTH'].fillna(median_poorhlth, inplace=True)
df['HHADULT'].fillna(median_hhadult, inplace=True)
df['INCOME3'].fillna(median_income, inplace=True)



# Display the DataFrame with imputed values
df.head()


,SEXVAR,HHADULT,PHYSHLTH,MENTHLTH,POORHLTH,PRIMINSR,MEDCOST1,EXERANY2,LASTDEN4,RMVTETH4,CVDSTRK3,ASTHMA3,CHCCOPD3,ADDEPEV3,HAVARTH4,EDUCA,RENTHOM1,EMPLOY1,INCOME3,Class
0,1.0,2.0,88.0,88.0,88.0,3.0,2.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,6.0,1.0,7.0,11.0,Mild CD
1,2.0,2.0,88.0,88.0,88.0,88.0,2.0,1.0,2.0,1.0,2.0,2.0,2.0,2.0,1.0,6.0,1.0,2.0,7.0,No CD
2,1.0,2.0,4.0,77.0,4.0,3.0,2.0,1.0,1.0,8.0,2.0,2.0,2.0,1.0,1.0,6.0,1.0,7.0,9.0,Mild CD
3,1.0,2.0,88.0,88.0,88.0,3.0,2.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,1.0,6.0,1.0,7.0,99.0,No CD
4,1.0,2.0,1.0,3.0,88.0,2.0,2.0,1.0,1.0,8.0,2.0,2.0,2.0,2.0,2.0,6.0,1.0,7.0,6.0,No CD


In [158]:
# Count blank values in each column
blank_values = df.isnull().sum()

# Print the number of blank values in each column
print("Number of blank values in each column:")
print(blank_values)


Number of blank values in each column:
SEXVAR      0
HHADULT     0
PHYSHLTH    0
MENTHLTH    0
POORHLTH    0
PRIMINSR    0
MEDCOST1    0
EXERANY2    0
LASTDEN4    0
RMVTETH4    0
CVDSTRK3    0
ASTHMA3     0
CHCCOPD3    0
ADDEPEV3    0
HAVARTH4    0
EDUCA       0
RENTHOM1    0
EMPLOY1     0
INCOME3     0
Class       0
dtype: int64


In [159]:
# Convert values in specified columns to int except for the last column "Class"
df[filtered_lav[:-1]] = df[filtered_lav[:-1]].astype(int)

df.head()


,SEXVAR,HHADULT,PHYSHLTH,MENTHLTH,POORHLTH,PRIMINSR,MEDCOST1,EXERANY2,LASTDEN4,RMVTETH4,CVDSTRK3,ASTHMA3,CHCCOPD3,ADDEPEV3,HAVARTH4,EDUCA,RENTHOM1,EMPLOY1,INCOME3,Class
0,1,2,88,88,88,3,2,1,1,1,2,2,2,2,2,6,1,7,11,Mild CD
1,2,2,88,88,88,88,2,1,2,1,2,2,2,2,1,6,1,2,7,No CD
2,1,2,4,77,4,3,2,1,1,8,2,2,2,1,1,6,1,7,9,Mild CD
3,1,2,88,88,88,3,2,1,1,1,2,2,2,2,1,6,1,7,99,No CD
4,1,2,1,3,88,2,2,1,1,8,2,2,2,2,2,6,1,7,6,No CD


In [160]:
# Print unique values in each column
for column in df.columns:
    print(f"Unique values in {column}:")
    print(df[column].unique())
    print()

Unique values in SEXVAR:
[1 2]

Unique values in HHADULT:
[ 2  1  4  3 10  6 99  5 77  9  7 17 12  8 16 13 40 22 23 18 14 11  0 88
 21 51 15]

Unique values in PHYSHLTH:
[88  4  1  3 10 20 77 30 14 15  2  5 25 29  7 13 99 28  8 17 12  6 21 19
 27 26 18 16  9 11 22 23 24]

Unique values in MENTHLTH:
[88 77  3  2  1 30  5 10 14 12 15 99  4  8  6 20  7 29  9 25 21 26 18 28
 23 16 11 27 13 17 24 22 19]

Unique values in POORHLTH:
[88  4  2 30 77 14  1 10  5  8 15 99  6 20  7  3 28 12 25  9 18 17 29 16
 22 26 21 27 24 19 13 23 11]

Unique values in PRIMINSR:
[ 3 88  2  1  7 10  5  9 77 99  4  8  6]

Unique values in MEDCOST1:
[2 1 7 9]

Unique values in EXERANY2:
[1 2 7 9]

Unique values in LASTDEN4:
[1 2 4 3 7 8 9]

Unique values in RMVTETH4:
[1 8 2 3 7 9]

Unique values in CVDSTRK3:
[2 7 1 9]

Unique values in ASTHMA3:
[2 1 7 9]

Unique values in CHCCOPD3:
[2 1 7 9]

Unique values in ADDEPEV3:
[2 1 7 9]

Unique values in HAVARTH4:
[2 1 7 9]

Unique values in EDUCA:
[6 5 4 2 3 9 1]

Unique

In [162]:
# Calculate the median excluding values 88, 99, and 77 for each column
hhadult_median = df['HHADULT'][~df['HHADULT'].isin([88, 99, 77])].median()
physhlth_median = df['PHYSHLTH'][~df['PHYSHLTH'].isin([88, 99, 77])].median()
menthlth_median = df['MENTHLTH'][~df['MENTHLTH'].isin([88, 99, 77])].median()
poorhlth_median = df['POORHLTH'][~df['POORHLTH'].isin([88, 99, 77])].median()
priminsr_median = df['PRIMINSR'][~df['PRIMINSR'].isin([88, 99, 77])].median()

# Replace values 88, 99, and 77 with the calculated median
df['HHADULT'] = df['HHADULT'].replace([88, 99, 77], hhadult_median)
df['PHYSHLTH'] = df['PHYSHLTH'].replace([88, 99, 77], physhlth_median)
df['MENTHLTH'] = df['MENTHLTH'].replace([88, 99, 77], menthlth_median)
df['POORHLTH'] = df['POORHLTH'].replace([88, 99, 77], poorhlth_median)
df['PRIMINSR'] = df['PRIMINSR'].replace([88, 99, 77], priminsr_median)

# Print unique values after replacement
print("Unique values in HHADULT after replacing 88, 99, and 77 with the median value:")
print(df['HHADULT'].unique())
print("\nUnique values in PHYSHLTH after replacing 88, 99, and 77 with the median value:")
print(df['PHYSHLTH'].unique())
print("\nUnique values in MENTHLTH after replacing 88, 99, and 77 with the median value:")
print(df['MENTHLTH'].unique())
print("\nUnique values in POORHLTH after replacing 88, 99, and 77 with the median value:")
print(df['POORHLTH'].unique())
print("\nUnique values in PRIMINSR after replacing 88, 99, and 77 with the median value:")
print(df['PRIMINSR'].unique())


Unique values in HHADULT after replacing 88, 99, and 77 with the median value:
[ 2  1  4  3 10  6  5  9  7 17 12  8 16 13 40 22 23 18 14 11  0 21 51 15]

Unique values in PHYSHLTH after replacing 88, 99, and 77 with the median value:
[ 7  4  1  3 10 20 30 14 15  2  5 25 29 13 28  8 17 12  6 21 19 27 26 18
 16  9 11 22 23 24]

Unique values in MENTHLTH after replacing 88, 99, and 77 with the median value:
[ 5  3  2  1 30 10 14 12 15  4  8  6 20  7 29  9 25 21 26 18 28 23 16 11
 27 13 17 24 22 19]

Unique values in POORHLTH after replacing 88, 99, and 77 with the median value:
[ 8  4  2 30 14  1 10  5 15  6 20  7  3 28 12 25  9 18 17 29 16 22 26 21
 27 24 19 13 23 11]

Unique values in PRIMINSR after replacing 88, 99, and 77 with the median value:
[ 3  2  1  7 10  5  9  4  8  6]


In [163]:
# List of columns to handle
columns_to_handle = ['CVDSTRK3', 'ASTHMA3', 'CHCCOPD3', 'ADDEPEV3', 'HAVARTH4', 'MEDCOST1', 'EXERANY2' ]

for column in columns_to_handle:
    # Calculate the mode or median value of the column, excluding the values 7 and 9
    replacement_value = df.loc[~df[column].isin([7, 9]), column].mode()[0]

    # Replace the values 7 and 9 with the calculated mode/median value
    df[column] = df[column].replace([7, 9], replacement_value)

    # Print unique values after replacement and conversion for each column
    print(f"Unique values in {column} after replacing 7 and 9 and converting to int:")
    print(df[column].unique())
    print()


Unique values in CVDSTRK3 after replacing 7 and 9 and converting to int:
[2 1]

Unique values in ASTHMA3 after replacing 7 and 9 and converting to int:
[2 1]

Unique values in CHCCOPD3 after replacing 7 and 9 and converting to int:
[2 1]

Unique values in ADDEPEV3 after replacing 7 and 9 and converting to int:
[2 1]

Unique values in HAVARTH4 after replacing 7 and 9 and converting to int:
[2 1]

Unique values in MEDCOST1 after replacing 7 and 9 and converting to int:
[2 1]

Unique values in EXERANY2 after replacing 7 and 9 and converting to int:
[1 2]



In [164]:
# List of columns to handle
columns_to_handle = ['LASTDEN4', 'RMVTETH4']

for column in columns_to_handle:
    # Calculate the median value of the column, excluding the values 7, 8, and 9
    replacement_value = df.loc[~df[column].isin([7, 8, 9]), column].median()

    # Replace the values 7, 8, and 9 with the calculated median value
    df[column] = df[column].replace([7, 8, 9], replacement_value)

    # Convert the column to integer type
    df[column] = df[column].astype(int)

    # Print unique values after replacement and conversion for each column
    print(f"Unique values in {column} after replacing 7, 8, and 9 and converting to int:")
    print(df[column].unique())
    print()


Unique values in LASTDEN4 after replacing 7, 8, and 9 and converting to int:
[1 2 4 3]

Unique values in RMVTETH4 after replacing 7, 8, and 9 and converting to int:
[1 2 3]



In [165]:
# Calculate the median value of the "EDUCA" column, excluding the value 9
replacement_value = df.loc[df['EDUCA'] != 9, 'EDUCA'].median()

# Replace the value 9 with the calculated median value
df['EDUCA'] = df['EDUCA'].replace(9, replacement_value)

# Convert the column to integer type
df['EDUCA'] = df['EDUCA'].astype(int)

# Print unique values after replacement and conversion
print("Unique values in EDUCA after replacing 9 with the median value and converting to int:")
print(df['EDUCA'].unique())


Unique values in EDUCA after replacing 9 with the median value and converting to int:
[6 5 4 2 3 1]


In [166]:
# Calculate the median value of the "RENTHOM1" column, excluding the values 7 and 9
replacement_value = df.loc[~df['RENTHOM1'].isin([7, 9]), 'RENTHOM1'].median()

# Replace the values 7 and 9 with the calculated median value
df['RENTHOM1'] = df['RENTHOM1'].replace([7, 9], replacement_value)

# Convert the column to integer type
df['RENTHOM1'] = df['RENTHOM1'].astype(int)

# Print unique values after replacement and conversion
print("Unique values in RENTHOM1 after replacing 7 and 9 with the median value and converting to int:")
print(df['RENTHOM1'].unique())


Unique values in RENTHOM1 after replacing 7 and 9 with the median value and converting to int:
[1 2 3]


In [167]:
# Calculate the median value of the "EMPLOY1" column, excluding the value 9
replacement_value = df.loc[df['EMPLOY1'] != 9, 'EMPLOY1'].median()

# Replace the value 9 with the calculated median value
df['EMPLOY1'] = df['EMPLOY1'].replace(9, replacement_value)

# Convert the column to integer type
df['EMPLOY1'] = df['EMPLOY1'].astype(int)

# Print unique values after replacement and conversion
print("Unique values in EMPLOY1 after replacing 9 with the median value and converting to int:")
print(df['EMPLOY1'].unique())


Unique values in EMPLOY1 after replacing 9 with the median value and converting to int:
[7 2 8 1 5 3 6 4]


In [168]:
# Calculate the median value of the "INCOME3" column, excluding the values 77 and 99
replacement_value = df.loc[~df['INCOME3'].isin([77, 99]), 'INCOME3'].median()

# Replace the values 77 and 99 with the calculated median value
df['INCOME3'] = df['INCOME3'].replace([77, 99], replacement_value)

# Convert the column to integer type
df['INCOME3'] = df['INCOME3'].astype(int)

# Print unique values after replacement and conversion
print("Unique values in INCOME3 after replacing 77 and 99 with the median value and converting to int:")
print(df['INCOME3'].unique())


Unique values in INCOME3 after replacing 77 and 99 with the median value and converting to int:
[11  7  9  6  2  5  8  1  4  3 10]


In [169]:
# Print unique values in each column
for column in df.columns:
    print(f"Unique values in {column}:")
    print(df[column].unique())
    print()

Unique values in SEXVAR:
[1 2]

Unique values in HHADULT:
[ 2  1  4  3 10  6  5  9  7 17 12  8 16 13 40 22 23 18 14 11  0 21 51 15]

Unique values in PHYSHLTH:
[ 7  4  1  3 10 20 30 14 15  2  5 25 29 13 28  8 17 12  6 21 19 27 26 18
 16  9 11 22 23 24]

Unique values in MENTHLTH:
[ 5  3  2  1 30 10 14 12 15  4  8  6 20  7 29  9 25 21 26 18 28 23 16 11
 27 13 17 24 22 19]

Unique values in POORHLTH:
[ 8  4  2 30 14  1 10  5 15  6 20  7  3 28 12 25  9 18 17 29 16 22 26 21
 27 24 19 13 23 11]

Unique values in PRIMINSR:
[ 3  2  1  7 10  5  9  4  8  6]

Unique values in MEDCOST1:
[2 1]

Unique values in EXERANY2:
[1 2]

Unique values in LASTDEN4:
[1 2 4 3]

Unique values in RMVTETH4:
[1 2 3]

Unique values in CVDSTRK3:
[2 1]

Unique values in ASTHMA3:
[2 1]

Unique values in CHCCOPD3:
[2 1]

Unique values in ADDEPEV3:
[2 1]

Unique values in HAVARTH4:
[2 1]

Unique values in EDUCA:
[6 5 4 2 3 1]

Unique values in RENTHOM1:
[1 2 3]

Unique values in EMPLOY1:
[7 2 8 1 5 3 6 4]

Unique values

In [170]:
# List of columns to handle
columns_to_handle = ['SEXVAR','MEDCOST1', 'EXERANY2', 'CVDSTRK3', 'ASTHMA3', 'CHCCOPD3', 'ADDEPEV3', 'HAVARTH4']

for column in columns_to_handle:
    # Replace the value 2 with 0 in each column
    df[column] = df[column].replace(2, 0)

    # Print unique values after replacement
    print(f"Unique values in {column} after replacing 2 with 0:")
    print(df[column].unique())
    print()


Unique values in SEXVAR after replacing 2 with 0:
[1 0]

Unique values in MEDCOST1 after replacing 2 with 0:
[0 1]

Unique values in EXERANY2 after replacing 2 with 0:
[1 0]

Unique values in CVDSTRK3 after replacing 2 with 0:
[0 1]

Unique values in ASTHMA3 after replacing 2 with 0:
[0 1]

Unique values in CHCCOPD3 after replacing 2 with 0:
[0 1]

Unique values in ADDEPEV3 after replacing 2 with 0:
[0 1]

Unique values in HAVARTH4 after replacing 2 with 0:
[0 1]



In [171]:
# Print unique values in each column
for column in df.columns:
    print(f"Unique values in {column}:")
    print(df[column].unique())
    print()

Unique values in SEXVAR:
[1 0]

Unique values in HHADULT:
[ 2  1  4  3 10  6  5  9  7 17 12  8 16 13 40 22 23 18 14 11  0 21 51 15]

Unique values in PHYSHLTH:
[ 7  4  1  3 10 20 30 14 15  2  5 25 29 13 28  8 17 12  6 21 19 27 26 18
 16  9 11 22 23 24]

Unique values in MENTHLTH:
[ 5  3  2  1 30 10 14 12 15  4  8  6 20  7 29  9 25 21 26 18 28 23 16 11
 27 13 17 24 22 19]

Unique values in POORHLTH:
[ 8  4  2 30 14  1 10  5 15  6 20  7  3 28 12 25  9 18 17 29 16 22 26 21
 27 24 19 13 23 11]

Unique values in PRIMINSR:
[ 3  2  1  7 10  5  9  4  8  6]

Unique values in MEDCOST1:
[0 1]

Unique values in EXERANY2:
[1 0]

Unique values in LASTDEN4:
[1 2 4 3]

Unique values in RMVTETH4:
[1 2 3]

Unique values in CVDSTRK3:
[0 1]

Unique values in ASTHMA3:
[0 1]

Unique values in CHCCOPD3:
[0 1]

Unique values in ADDEPEV3:
[0 1]

Unique values in HAVARTH4:
[0 1]

Unique values in EDUCA:
[6 5 4 2 3 1]

Unique values in RENTHOM1:
[1 2 3]

Unique values in EMPLOY1:
[7 2 8 1 5 3 6 4]

Unique values

In [172]:
df.head()

,SEXVAR,HHADULT,PHYSHLTH,MENTHLTH,POORHLTH,PRIMINSR,MEDCOST1,EXERANY2,LASTDEN4,RMVTETH4,CVDSTRK3,ASTHMA3,CHCCOPD3,ADDEPEV3,HAVARTH4,EDUCA,RENTHOM1,EMPLOY1,INCOME3,Class
0,1,2,7,5,8,3,0,1,1,1,0,0,0,0,0,6,1,7,11,Mild CD
1,0,2,7,5,8,3,0,1,2,1,0,0,0,0,1,6,1,2,7,No CD
2,1,2,4,5,4,3,0,1,1,1,0,0,0,1,1,6,1,7,9,Mild CD
3,1,2,7,5,8,3,0,1,1,1,0,0,0,0,1,6,1,7,7,No CD
4,1,2,1,3,8,2,0,1,1,1,0,0,0,0,0,6,1,7,6,No CD


In [173]:
df.shape

(63630, 20)

In [175]:

from google.colab import files
df.to_csv('lavanya-apr-20.csv', index=True)
files.download('lavanya-apr-20.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>